In [101]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from collections import defaultdict

In [2]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

In [21]:
# Reading the data
dataset = pd.read_json('renttherunway_final_data.json.gz', lines=True)
dataset = dataset.dropna()

# 0 = Small, 1 = Fit, 2 = Large
dataset.loc[dataset["fit"] == "small", "fit"] = 0

dataset.loc[dataset["fit"] == "fit", "fit"] = 1

dataset.loc[dataset["fit"] == "large", "fit"] = 2

In [102]:
# Converting the data into list of dictionaries
data = pd.DataFrame(dataset).to_dict('record')

for d in data:
    d['weight'] = int(d['weight'].split('lbs')[0])
    if(int(d['height'].split(' ')[1].split('"')[0])<10):
        height2 = int(d['height'].split(' ')[1].split('"')[0])*10
    else:
        height2 = int(d['height'].split(' ')[1].split('"')[0])
    d['height'] = int(d['height'][0][0])*100+height2
    
####converting categroies type to one hot
catogeries = ['rented for','body type']
for cat in catogeries:
    categories_list = defaultdict(int)
    for d in data:
        categories_list[d[cat]] += 1
        
    categories_id = defaultdict(int)

    i = 0
    for cID in  categories_list:
        categories_id[cID] = i
        i+=1
    for d in data:
        f = [0]*len(categories_list)
        f[categories_id[d[cat]]] = 1
        d[cat] = f[:len(categories_list)-1]

In [ ]:
# Extracting features from the data
def feature(d):
    f = [1, d['rating'], len(d['review_text']), len(d['review_summary'])]
    return np.array(f)

In [265]:
# Add features
df = []
for d in data:
    arr = []
#     arr.append(d['weight'])
    arr.append(d['rating'])
    arr += d['rented for']
    arr.append(len(d['review_text']))
    arr += d['body type']
#     arr.append(d['height'])
    arr.append(d['size'])
#     arr.append(d['age'])
    df.append(arr)

In [266]:
y = dataset["fit"]

y_cat = to_categorical(y)

# Split data
test_y = y_cat[int(len(y_cat) * .8):]
train_y = y_cat[:int(len(y_cat) * .8)]
test_f = df[int(len(y_cat) * .8):]
train_f = df[:int(len(y_cat) * .8)]

# Convert to numpy array.
test_f = np.array(test_f)
train_f = np.array(train_f)

In [267]:
model = Sequential()
model.add(Dense(60, input_shape = (len(train_f[0]), ), activation = "relu"))
model.add(Dense(40, activation = "relu"))
model.add(Dropout(0.1))
model.add(Dense(3, activation = "softmax"))
model.compile(Adam(lr = 0.001), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_99 (Dense)             (None, 60)                1080      
_________________________________________________________________
dense_100 (Dense)            (None, 40)                2440      
_________________________________________________________________
dropout_28 (Dropout)         (None, 40)                0         
_________________________________________________________________
dense_101 (Dense)            (None, 3)                 123       
Total params: 3,643
Trainable params: 3,643
Non-trainable params: 0
_________________________________________________________________


In [268]:
model.fit(train_f, train_y, verbose=1, epochs=10)

Epoch 1/10
117104/117104 [==============================] - 3s 28us/step - loss: 0.8589 - accuracy: 0.7276
Epoch 2/10
117104/117104 [==============================] - 3s 26us/step - loss: 0.7409 - accuracy: 0.7356
Epoch 3/10
117104/117104 [==============================] - 3s 26us/step - loss: 0.7372 - accuracy: 0.7357
Epoch 4/10
117104/117104 [==============================] - 3s 27us/step - loss: 0.7331 - accuracy: 0.7357
Epoch 5/10
117104/117104 [==============================] - 3s 28us/step - loss: 0.7293 - accuracy: 0.7357
Epoch 6/10
117104/117104 [==============================] - 4s 34us/step - loss: 0.7278 - accuracy: 0.7358
Epoch 7/10
117104/117104 [==============================] - 3s 27us/step - loss: 0.7260 - accuracy: 0.7362
Epoch 8/10
117104/117104 [==============================] - 3s 27us/step - loss: 0.7222 - accuracy: 0.7361
Epoch 9/10
117104/117104 [==============================] - 3s 28us/step - loss: 0.7211 - accuracy: 0.7361
Epoch 10/10
117104/117104 [==========

In [269]:
# Confusion matrix
y_pred_class = model.predict_classes(test_f)
from sklearn.metrics import confusion_matrix
y_pred = model.predict(test_f)
y_test_class = np.argmax(test_y, axis=1)
confusion_matrix(y_test_class, y_pred_class)

array([[   91,  3840,    11],
       [   82, 21475,    22],
       [   38,  3697,    21]])

In [270]:
# Classification report.
from sklearn.metrics import classification_report
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       0.43      0.02      0.04      3942
           1       0.74      1.00      0.85     21579
           2       0.39      0.01      0.01      3756

    accuracy                           0.74     29277
   macro avg       0.52      0.34      0.30     29277
weighted avg       0.65      0.74      0.63     29277

